<a href="https://colab.research.google.com/github/ikjot-2605/Interview_Pairing_Automate/blob/main/Interview_Pairings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

import pandas as pd

In [61]:
sh = gc.open('WEC Mock Interview pairings')
worksheet_list = sh.worksheets()
worksheet_list_length = len(worksheet_list)
all_interviewees = set()
all_interviewers = set()
for i in range(0,worksheet_list_length):
  current_worksheet = worksheet_list[i]
  rows = current_worksheet.get_all_values()
  pd_val = pd.DataFrame.from_records(rows)
  for index, row in pd_val.iterrows():
    if index>1:
      all_interviewers.add(row[1])
      all_interviewees.add(row[0])
interviewee_list = list(all_interviewees)
interviewer_list = list(all_interviewers)
matrix = {}

for i in range (0,len(interviewer_list)):
  matrix[interviewer_list[i]] = [];
  for j in range (0,len(interviewee_list)):
    matrix[interviewer_list[i]].append(interviewee_list[j])

for i in range (0,len(interviewer_list)):
  for j in range(0,worksheet_list_length):
    current_worksheet = worksheet_list[j]
    rows = current_worksheet.get_all_values()
    pd_val = pd.DataFrame.from_records(rows)
    for index, row in pd_val.iterrows():
      if row[1]==interviewer_list[i]:
        matrix[interviewer_list[i]].remove(row[0])
print(all_interviewers)



  

{'Krithik Vaidya(WEC)', 'Abhinav(WEC)', 'Krishna Swaroop (WEC)', 'Mohit(WEC)', 'Shashank(WEC)', 'Jaidev Chittoria(WEC)', 'Niranjan S Y(WEC)', 'Sai Krishna Anand (WEC)', 'Shreeraksha(WEC)', 'Kevin Joe(WEC)', 'Manav Singhal(WEC)', 'Sai Krishna Anand(WEC)', 'Vaibhav Puri(WEC)', 'Arpitha R(WEC)', 'Aditya Jain(WEC)', 'Suhas K S(WEC)', 'Aniruddh Patil(WEC)', 'Adharsh Kamath(WEC)', 'Sriram Rao(WEC)', 'Arnav Nair(WEC)', 'Shruthan(WEC)', 'Kinshuk Kashyap(WEC)', 'Akashdeep S(WEC)', 'Abhishek Kumar(WEC)', 'Saurabh Agarwala(WEC)'}


In [62]:
two_interviewer_list = []
current_worksheet = worksheet_list[worksheet_list_length-1]
rows = current_worksheet.get_all_values()
pd_val = pd.DataFrame.from_records(rows)
for index, row in pd_val.iterrows():
  two_interviewer_list.append(row[1])
two_interviewer_list.pop(0)
two_interviewer_list.pop(0)  
two_interview_list = []
one_interview_list = []
for i in range (0,len(two_interviewer_list)):
  if i>len(two_interviewer_list)-1 : break
  count_of = two_interviewer_list.count(two_interviewer_list[i])
  if count_of ==1 :
    one_interview_list.append(two_interviewer_list[i])
  if count_of==2 and two_interview_list.count(two_interviewer_list[i])==0:   
    two_interview_list.append(two_interviewer_list[i])


In [70]:
allotments = []
interviewees_allotted = set();
for i in range (0,len(two_interview_list)):
  for j in range (0,len(matrix[two_interview_list[i]])):
    if matrix[two_interview_list[i]][j] in interviewees_allotted:
      continue
    else: 
      interviewees_allotted.add(matrix[two_interview_list[i]][j])
      allotments.append([matrix[two_interview_list[i]][j],two_interview_list[i]])
      break  
for i in range (0,len(one_interview_list)):
  count = 0
  for j in range (0,len(matrix[one_interview_list[i]])):
    if count==2:break
    if matrix[one_interview_list[i]][j] in interviewees_allotted:
      continue
    else: 
      interviewees_allotted.add(matrix[one_interview_list[i]][j])
      allotments.append([matrix[one_interview_list[i]][j],one_interview_list[i]])
      count = count+1

print(allotments)
print(len(interviewees_allotted))
import csv

with open("output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(allotments)

[['Ali Murtaza(WEC)', 'Arpitha R(WEC)'], ['Adithya Rajesh(WEC)', 'Aditya Jain(WEC)'], ['Srujan Bharadwaj(WEC)', 'Abhinav(WEC)'], ['Sushanth Sathesh Rao(WEC)', 'Aniruddh Patil(WEC)'], ['saurabh gole(WEC)', 'Adharsh Kamath(WEC)'], ['Chaithanya Shyam D(WEC)', 'Saurabh Agarwala(WEC)'], ['Janmansh Agarwal(WEC)', 'Shruthan(WEC)'], ['Rakshit P(WEC)', 'Akashdeep S(WEC)'], ['Tharun K(WEC)', 'Jaidev Chittoria(WEC)'], ['Rakshita Varadarajan(WEC)', 'Jaidev Chittoria(WEC)'], ['Gaurav Singh(WEC)', 'Kevin Joe(WEC)'], ['Ikjot Singh Dhody(WEC)', 'Kevin Joe(WEC)'], ['Balajinaidu V(WEC)', 'Krishna Swaroop (WEC)'], ['K Dharmick Sai(WEC)', 'Krishna Swaroop (WEC)'], ['Saish Mendke(WEC)', 'Niranjan S Y(WEC)'], ['Sudesh Gowda J(WEC)', 'Niranjan S Y(WEC)'], ['Achintya Kumar(WEC)', 'Manav Singhal(WEC)'], ['Addhyan Malhotra(WEC)', 'Manav Singhal(WEC)'], ['Aritra Sinha(WEC)', 'Suhas K S(WEC)'], ['yash sharma(WEC)', 'Suhas K S(WEC)'], ['Nanda Kishore(WEC)', 'Shashank(WEC)'], ['Tanmay Ranaware(WEC)', 'Shashank(WEC)